In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers import Activation, Dense, Dropout
from keras.callbacks import ModelCheckpoint

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
#Config parameter
time_step = 20
future_time = 1
rate = 30
hidden_dim = 50
dropout_rate = 0.2
loss_fn = 'mean_squared_error'
opt = 'adam'
lstm = LSTM

2.13.0


In [ ]:
#đọc file, tính Delta giữu giá cổ phiếu max và min
path_csv = 'MSN.csv'
data_read = pd.read_csv(path_csv)
Close_price_max = data_read['Close'].max()
Close_price_min = data_read['Close'].min()
delta = float(Close_price_max - Close_price_min)

In [ ]:
data_read.index = pd.to_datetime(data_read['Date/Time'])
start=data_read['Date/Time'][0]
time_str_start= start[0:len(start)-5] + ' 9:00'
end=data_read['Date/Time'][len(data_read['Date/Time'])-1]
time_str_end = end[0:len(start)-5] + ' 14:59'
new_index_update = pd.date_range(start=time_str_start, end= time_str_end, freq='min')
data_read = data_read.reindex(new_index_update)
data_read = data_read.rename_axis('Date')
data_read_new = data_read.drop(['Ticker', 'Date/Time', 'Open', 'High', 'Low', 'Open Interest'], axis=1)
data_read_new_update = data_read_new.fillna(0)

In [ ]:
def MinToOtherTime(in_data,rate):
    data = in_data.copy()
    count = 0
    num_of_value_0 = 0
    value_close = 0
    value_volume = 0
    Close =[]
    Volume = []
    for i in range(len(data)):
        if data['Close'][i] == 0:
            num_of_value_0  = num_of_value_0 + 1
        value_close += data['Close'][i]
        value_volume += data['Volume'][i]
        count +=1
        if count == rate and num_of_value_0 == rate:
            mean_value_close = 0
            value_volume = 0
            Close.append(mean_value_close)
            Volume.append(value_volume)
            count = 0
            num_of_value_0 = 0

        elif count == (rate) and count != num_of_value_0:
            mean_value_close = value_close/(count - num_of_value_0)
            num_of_value_0 = 0
            Close.append(mean_value_close)
            Volume.append(value_volume)
            value_close = 0
            value_volume = 0
            count = 0
    df = pd.DataFrame({'Close':Close, 'Volume':Volume})
    return df

In [ ]:
data_trasform = MinToOtherTime(data_read_new_update, rate)
data_use_train = data_trasform[~(data_trasform == 0).all(axis=1)]

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
values = scaler.fit_transform(data_use_train.values)
df_values = pd.DataFrame(values, columns =['Close', 'Volume'])

In [ ]:
def create_data_train(df_values, time_step):
    in_data = df_values.copy()
    cols_list_new = []
    colunms = ['Close', 'Volume']
    cols_list_new = []
    for colunm in colunms:
        i = 1
        while i < time_step:
            in_data[colunm + " +{}".format(i)] = in_data[colunm].shift(-i)
            i = i + 1
    # in_data["target"] = in_data["Close"].shift(-time_step - future_time + 1)
    result = in_data.dropna(axis=0)
    cols_list = result.columns.tolist()
    cols_list_new = cols_list[0:1] + cols_list[2:time_step+1] + cols_list[1:2] +  cols_list[time_step+1:]
    out_data = result[cols_list_new]
    return out_data

In [ ]:
df_train = create_data_train(df_values, time_step)

In [ ]:
#lấy giá trị cổ phiếu hiện tại và đôi chiều tập x
x_test = df_train.values
y_present = float(x_test[time_step-1:time_step].item(0))
x_test = x_test.reshape((x_test.shape[0],time_step,-1))

In [ ]:
#tạo model
x = Input(name='input', shape=(time_step, 2), dtype='float32')
lstm_1_test = lstm(hidden_dim, input_shape=(time_step, 2),
              return_sequences=True, name='lstm_1_test')(x)
dropout_1_test = Dropout(dropout_rate, name = 'dropout_1_test')(lstm_1_test)
lstm_2_test = lstm(100, input_shape=(time_step, 2),
              return_sequences=False, name='lstm_2_test')(dropout_1_test)
dropout_2_test = Dropout(dropout_rate, name = 'dropout_2_test')(lstm_2_test)
dense_1_test = Dense(50, activation='relu', name='dense_1_test')(dropout_2_test)
y = Dense(1, activation='linear', name='dense_2')(dense_1_test)
model_test = Model(inputs=x, outputs=y, name = 'Model_test')
model_test.load_weights('weights.best.hdf5')
model_test.compile(loss=loss_fn, optimizer=opt)

In [ ]:
#Hàm tính biến động giá
def predict_fluctuate(x_test):
    y_pred = float(model_test.predict(x_test).item(0))
    fluctuate = (y_pred - y_present)*delta
    return fluctuate

In [ ]:
print('Bien dong gia sau {} phut sau la {:.2f}'.format(rate, predict_fluctuate(x_test)))